In [1]:
import math
from pathlib import Path

import pickle as pkl
from datetime import datetime

import pymysql

start = datetime.now()

In [2]:
property = {
    "host": "",
    "user": "",
    "password": "",
    "database": "",
}

In [3]:
def sql(query, method = "all"):
    with connection.cursor() as cursor:
        cursor.execute(query)
        if method == "all":
            result = cursor.fetchall()
        elif method == "one":
            result = cursor.fetchone()
        else:
            ValueError("query error:", query)
    end = datetime.now()
    time_taken = end - start
    print(f"query completed in {time_taken}")

    return result

In [4]:
dms_ids_c = {
    "A": 7,
    "B": 7,
    "C": 7,
    "I": 8,
    "L": 8,
    "R": 9,
    "S": 7,
}
dms_p_count = 28

z_fields = [f"z_{prefix}{str(num).zfill(2)}" for prefix, max_num in dms_ids_c.items() for num in range(1, max_num + 1)]
z_fields += [f"z_D{str(num).zfill(2)}" for num in range(1, dms_p_count + 1)]
z_fields_str = ", ".join(z_fields)
print(z_fields_str)
print()

sample_fields = f"tgu_id, idno, cid, invoice, industry_id, position_id, sm, sy, finish_date, {z_fields_str}".split(", ")
print(sample_fields)

z_A01, z_A02, z_A03, z_A04, z_A05, z_A06, z_A07, z_B01, z_B02, z_B03, z_B04, z_B05, z_B06, z_B07, z_C01, z_C02, z_C03, z_C04, z_C05, z_C06, z_C07, z_I01, z_I02, z_I03, z_I04, z_I05, z_I06, z_I07, z_I08, z_L01, z_L02, z_L03, z_L04, z_L05, z_L06, z_L07, z_L08, z_R01, z_R02, z_R03, z_R04, z_R05, z_R06, z_R07, z_R08, z_R09, z_S01, z_S02, z_S03, z_S04, z_S05, z_S06, z_S07, z_D01, z_D02, z_D03, z_D04, z_D05, z_D06, z_D07, z_D08, z_D09, z_D10, z_D11, z_D12, z_D13, z_D14, z_D15, z_D16, z_D17, z_D18, z_D19, z_D20, z_D21, z_D22, z_D23, z_D24, z_D25, z_D26, z_D27, z_D28

['tgu_id', 'idno', 'cid', 'invoice', 'industry_id', 'position_id', 'sm', 'sy', 'finish_date', 'z_A01', 'z_A02', 'z_A03', 'z_A04', 'z_A05', 'z_A06', 'z_A07', 'z_B01', 'z_B02', 'z_B03', 'z_B04', 'z_B05', 'z_B06', 'z_B07', 'z_C01', 'z_C02', 'z_C03', 'z_C04', 'z_C05', 'z_C06', 'z_C07', 'z_I01', 'z_I02', 'z_I03', 'z_I04', 'z_I05', 'z_I06', 'z_I07', 'z_I08', 'z_L01', 'z_L02', 'z_L03', 'z_L04', 'z_L05', 'z_L06', 'z_L07', 'z_L08', 'z_R01

In [5]:
def get_connection():
    return pymysql.connect(
        host=property["host"],
        user=property["user"],
        password=property["password"],
        database=property["database"],
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor,
    )

In [6]:
connection = get_connection()

try:
    tgu_str = f"tgu_id, idno, finish_date, {z_fields_str}"
    query = f"""
        SELECT {tgu_str}
        FROM target_user_sample
    """
    tgus = sql(query)
except Exception:
    connection.close()
finally:
    connection.close()

query completed in 0:01:36.374669


In [7]:
connection = get_connection()

try:
    query = """
        SELECT id_no, invoice, ind_cat_no, job_cat_no, start_date, end_date
        FROM exp_job
        WHERE job_cat_no > 0 OR ind_cat_no > 0
    """
    exps = sql(query)
except Exception:
    connection.close()
finally:
    connection.close()

query completed in 0:07:37.465491


In [8]:
connection = get_connection()

try:
    query = "SELECT id_no, update_date FROM basic"
    basics = sql(query)
except Exception:
    connection.close()
finally:
    connection.close()

query completed in 0:09:12.029137


In [9]:
connection = get_connection()

try:
    query = f"""SELECT cid, invoice FROM cid_mapping ORDER BY invoice, cid"""
    cids = sql(query)
except Exception:
    connection.close()
finally:
    connection.close()

query completed in 0:09:12.697715


In [10]:
print(len(tgus))
print(len(exps))
print(len(basics))
print(len(cids))

765621
18140719
8173047
47179


In [11]:
def convert2map(table_l):
    print(len(table_l))
    table_map = {}
    for i, item in enumerate(table_l):
        new_item = item.copy()
        if "idno" in new_item:
            key = "idno"
            idno = new_item[key]
            del new_item[key]
            if idno not in table_map:
                table_map[idno] = [new_item]
            else:
                table_map[idno].append(new_item)
        elif "id_no" in new_item:
            key = "id_no"
            idno = new_item[key]
            del new_item[key]
            if idno not in table_map:
                table_map[idno] = [new_item]
            else:
                table_map[idno].append(new_item)
        elif "invoice" in new_item:
            key = "invoice"
            invoice = new_item[key]
            if invoice not in table_map:
                del new_item[key]
                table_map[invoice] = new_item

    return table_map

In [12]:
tgu_map = convert2map(tgus)
del tgus
basic_map = convert2map(basics)
del basics
cid_map = convert2map(cids)
del cids
exp_map = convert2map(exps)
del exps

765621
8173047
47179
18140719


In [13]:
sample_datas = []
for i, (idno, tgu_obj) in enumerate(tgu_map.items(), start=1):
    if idno not in exp_map:
        continue

    exps = exp_map[idno]

    for exp in exps:
        start_ts = exp["start_date"].timestamp()
        start_y = int(exp["start_date"].strftime("%Y"))
        start_m = int(exp["start_date"].strftime("%m"))

        if exp["end_date"] and start_ts < exp["end_date"].timestamp():
            end_ts = exp["end_date"].timestamp()
            end_y = int(exp["end_date"].strftime("%Y"))
            end_m = int(exp["end_date"].strftime("%m"))
        else:
            if not exp["end_date"]:
                print(f"~~~ -> {exp['end_date']}")

            basic = basic_map[idno][0]
            end_ts = basic["update_date"].timestamp()
            end_y = int(basic["update_date"].strftime("%Y"))
            end_m = int(basic["update_date"].strftime("%m"))

        sm = (end_y - start_y) * 12 + (end_m - start_m)
        sm = 750 if sm > 750 else sm

        sy = -9.9999999999
        if exp["end_date"] and start_ts < exp["end_date"].timestamp():
            if sm == 0:
                sy = round(2 + math.log2(0.5 / 3), 10)
            else:
                sy = round(2 + math.log2(sm / 3), 10)

        tgus = tgu_obj

        min_diff = 1e20
        for tgu in tgus:
            cur_diff = abs((start_ts + end_ts) / 2 - tgu["finish_date"].timestamp())
            if cur_diff < min_diff:
                target_tgu = tgu
                min_diff = cur_diff

        invoice = int(exp["invoice"])
        cid = {"cid": 0} if invoice == 0 or invoice not in cid_map else cid_map[invoice]
        sample_data = target_tgu.copy()
        sample_data["idno"] = idno
        sample_data["cid"] = cid["cid"]
        sample_data["invoice"] = int(exp["invoice"])
        sample_data["industry_id"] = exp["ind_cat_no"]
        sample_data["position_id"] = exp["job_cat_no"]
        sample_data["sm"] = sm
        sample_data["sy"] = sy
        sample_datas.append(sample_data)

    if i % 100000 == 0 or i == len(exp_map):
        end = datetime.now()
        time_taken = end - start
        print(f"{i} completed in {time_taken}")

100000 completed in 0:10:13.098883
200000 completed in 0:10:20.582791
300000 completed in 0:10:28.054503
400000 completed in 0:10:35.470221
500000 completed in 0:10:42.853857
700000 completed in 0:10:55.957782


In [14]:
display_fields = ("tgu_id", "idno", "cid", "invoice", "industry_id", "position_id", "sm", "sy", "finish_date", "z_D01", "z_A01")
end = datetime.now()
time_taken = end - start
print(f"{i} completed in {time_taken}")
print(len(sample_datas))

700122 completed in 0:10:55.971685
2214509
